## Initialization

In [1]:
FN = 'predict'

In [2]:
import os
os.environ['THEANO_FLAGS'] = 'device=cpu,floatX=float32'

In [3]:
import theano
DEBUG = False
if DEBUG:
    import warnings
    warnings.filterwarnings('ignore', '.*Warning: cannot compute test val')
    warnings.filterwarnings('ignore', '.*has no test value')
    theano.config.optimizer = 'None'
    theano.config.compute_test_value = 'warn'

In [4]:
import keras
keras.__version__

/Users/danemichaelhankamer/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.2.4'

### No clipping of the indexes of the words to vocab_size

In [5]:
FN0 = 'vocabulary-embedding'

In [6]:
FN1 = 'train'

In [7]:
maxlend = 50
maxlenh = 25
maxlen = maxlend + maxlenh
rnn_size = 512
rnn_layers = 3 # match FN1
batch_norm = False

In [8]:
activation_rnn_size = 40 if maxlend else 0

### Initialize training parameter values for model

In [9]:
seed = 42
p_W, p_U, p_dense, weight_decay = 0, 0, 0, 0
optimizer = 'adam'
LR = 1e-4
batch_size = 64
# nflips = 10

### Set the train and test set

In [10]:
nb_train_samples = 30000
nb_val_samples = 3000

# Read word embedding from dataset

In [31]:
# import _pickle as cPickle
import pickle
import codecs # codecs.open
import numpy as np

with open('data.pkl', 'rb') as fp:
# with open('data.pkl', 'r+', encoding="utf-8") as fp:
    # embedding, idx2word, word2idx, glove_idx2idx
    embedding, idx2word, word2idx = pickle.load(fp, encoding='latin1')
# vocab_size, embedding_size
embedding_size = np.shape(embedding)
vocab_size = np.shape(embedding)

with open('data/%s.data.pkl'%FN0, 'rb') as fp:
    X, Y = cPickle.load(fp)

In [32]:
nb_unknown_words = 10

In [30]:
# print('number of examples', len(X), len(Y))
# print('dimension of embedding space for words', embedding_size)
# print('vocabulary size', vocab_size)
# print('the last %d words are holders for unk/oov words'%nb_unknown_words)
# print('total number of different words', len(idx2word), len(word2idx))
# print('number of words outside vocab... substitute using glove sim', len(glove_idx2idx))
# print('number of unknown/oov words', len(idx2word)-vocab_size-len(glove_idx2idx))
print(embedding)
print(idx2word)
print(word2idx)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
for i in range(nb_unknown_words):
    idx2word[vocab_size-1-i] = '<%d>'%i

TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

In [ ]:
for i in range(vocab_size - nb_unknown_words, len(idx2word)):
    idx2word[i] = idx2word[i] + '^'

oov0 = vocab_size - nb_unknown_words

for i in range(oov0, len(idx2word)):
    idx2word[i] = idx2word[i] + '^'

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=nb_val_samples, random_state=seed)
len(X_train), len(Y_train), len(X_test), len(Y_test)

del X
del Y

In [ ]:
empty = 0
eos = 1
idx2word[empty] = '_'
idx2word[eos] = '~'

In [20]:
import numpy as np
from keras.preprocessing import sequence
from keras.utils import np_utils
import random, sys

### Print an example of train and test set for some index

In [22]:
def prt(label, x):
    print(label+':')
    for w in x:
        print(idx2word[w]),
    print

i = 334
prt('H', Y_train[i])
prt('D', X_train[i])

i = 334
prt('H', Y_test[i])
prt('D', X_test[i])

# Model Formulation

In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
# from keras.layers.wrappers import TimeDistributed
from keras.layers.core import Lambda
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
import keras.backend as K

/Users/danemichaelhankamer/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Seed weight initialization

In [ ]:
random.seed(seed)
np.random.seed(seed)

In [ ]:
regularizer = l2(weight_decay) if weight_decay else None

### Build RNN, standard stacked LSTM with dropout

In [ ]:
rnn_model = Sequential()
if DEBUG:
    from keras.layers import InputLayer
    model_input = InputLayer(input_shape=(maxlen,))
    rnn_model.add(model_input)
    rnn_model.output.tag.test_value = np.random.randint(vocab_size, size=(batch_size, maxlen)).astype('float32')
    
rnn_model.add(Embedding(vocab_size, embedding_size,
                    input_length=maxlen,
                    embeddings_regularizer=regularizer,
                    weights=[embedding], mask_zero=True,
                    name='embedding_1'))

for i in range(rnn_layers):
    lstm = LSTM(rnn_size, return_sequences=True,
                kernel_regularizer=regularizer,
                recurrent_regularizer=regularizer,
                bias_regularizer=regularizer, dropout=p_W,
                recurrent_dropout=p_U, name='lstm_%d'%(i+1))
    rnn_model.add(lstm)
    rnn_model.add(Dropout(p_dense, name='dropout_%d'%(i+1)))

In [ ]:
if DEBUG:
    print rnn_model.output.tag.test_value.shape

### Load weights

In [ ]:
rnn_model.load_weights('data/%s.hdf5'%FN1, by_name=True)

In [ ]:
import h5py
with h5py.File('data/%s.hdf5'%FN1, mode='r') as f:
    if 'layer_names' not in f.attrs and 'model_weights' in f:
        f = f['model_weights']
    weights = [np.copy(v) for v in f['timedistributed_1'].itervalues()]

In [ ]:
map(lambda x: x.shape, weights)


### Headline model, special layer to reduce input to just the headline

In [ ]:
def simple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:, :maxlend, :], X[:, maxlend:, :]
    head_activations, head_words = head[:, :, :n], head[:, :, n:]
    desc_activations, desc_words = desc[:, :, :n], desc[:, :, n:]
    
    # activation for every head and desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=(2,2))
    ignore description words that are masked out #
    activation_energies = activation_energies + -1e20 * K.expand_dims(1.-K.cast(mask[:, :maxlend], 'float32'), 1)
    
    # compute desc word weights for every head word
    activation_energies = K.reshape(activation_energies, (-1, maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights, (-1, maxlenh, maxlend))
    
    # compute weighted average of desc words for every head word
    desc_avg_word = K.batch_dot(activation_weights, desc_words, axes=([2],[1]))
    return K.concatenate((desc_avg_word, head_words))

### Add model special layer above and models for TD and Activation

In [ ]:
model = Sequential()
model.add(rnn_model)

if activation_rnn_size:
    model.add(Lambda(simple_context, mask=lambda inputs, mask: mask[:, maxlend:],
                     output_shape = lambda input_shape: (input_shape[0], maxlenh, 2*(rnn_size - activation_rnn_size)),
                     name = 'simplecontext_1'))
    #model.add(TimeDistributed(Dense(vocab_size,
    #                                kernel_regularizer=regularizer,
    #                                bias_regularizer=regularizer,
    #                                name = 'timedistributed_1')))
    #model.add(Activation('softmax', name='activation_1'))

### Run model with optimizer and get summary

In [ ]:
from keras.optimizers import Adam, RMSprop 
# might need to change to 'RMSprop'... article uses this
# opt = Adam(lr=LR) or use Adam optimizer and reduce learning rate
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

K.set_value(model.optimizer.lr, np.float32(LR))

model.summary()

In [ ]:
n = 2 * (rnn_size - activation_rnn_size) # 944

### Out own softmax

In [ ]:
def output2probs(output):
    output = np.dot(output, weights[0]) + weights[1]
    output -= output.max()
    output = np.exp(output)
    output /= output.sum()
    return output

In [ ]:
def output2probs1(output):
    output0 = np.dot(output[:n//2], weights[0][:n//2,:])
    output1 = np.dot(output[n//2:], weights[0][n//2:,:])
    output = output0 + output1 # + output0 * output1
    output += weights[1]
    output -= output.max()
    output = np.exp(output)
    output /= output.sum()
    return output

# Test Model

In [ ]:
def lpadd(x, maxlend=maxlend, eos=eos):
    # left pre-pad a description to maxlend and add eos
    # the eos is the input to predicting the first word in the headline
    assert maxlend >= 0
    if maxlend == 0:
        return [eos]
    n = len(x)
    if n > maxlend:
        x = x[-maxlend:]
        n = maxlend
    return [empty] * (maxlend-n) + x + [eos] 

In [ ]:
samples = [lpadd([3]*26)]
# pad from right so the first maxlend will be desc followed by headline
data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty
                              padding='post', truncating='post')

In [ ]:
# returns true if padded properly
np.all(data[:, maxlend] == eos)

In [ ]:
data.shape, map(len, samples)

In [ ]:
probs = model.predict(data, verbose=0, batch_size=1)
probs.shape

# Sample Generation

In [ ]:
def beamsearch(predict, start=[empty]*maxlend + [eos], avoid=None, avoid_score=1,
               k=1, maxsample=maxlen, use_unk=True, oov=vocab_size-1, empty=empty, eos=eos, temperature=1.0):
    """return k samples (beams) and their NLL scores, each sample is a sequence of labels,
    all samples starts with an `empty` label and end with `eos` or truncated to length of `maxsample`.
    You need to supply `predict` which returns the label probability of each sample.
    `use_unk` allow usage of `oov` (out-of-vocabulary) label in samples
    """
    def sample(energy, n, temperature=temperature):
        """sample at most n different elements according to their energy"""
        n = min(n,len(energy))
        prb = np.exp(-np.array(energy) / temperature )
        res = []
        for i in xrange(n):
            z = np.sum(prb)
            r = np.argmax(np.random.multinomial(1, prb/z, 1))
            res.append(r)
            prb[r] = 0. # make sure we select each element only once
        return res

    dead_samples = []
    dead_scores = []
    live_samples = [list(start)]
    live_scores = [0]

    while live_samples:
        # for every possible live sample calc prob for every possible label 
        probs = predict(live_samples, empty=empty)
        assert vocab_size == probs.shape[1]

        # total score for every sample is sum of -log of word prb
        cand_scores = np.array(live_scores)[:,None] - np.log(probs)
        cand_scores[:,empty] = 1e20
        if not use_unk and oov is not None:
            cand_scores[:,oov] = 1e20
        if avoid:
            for a in avoid:
                for i, s in enumerate(live_samples):
                    n = len(s) - len(start)
                    if n < len(a):
                        # at this point live_sample is before the new word,
                        # which should be avoided, is added
                        cand_scores[i,a[n]] += avoid_score
        live_scores = list(cand_scores.flatten())
        

        # find the best (lowest) scores we have from all possible dead samples and
        # all live samples and all possible new words added
        scores = dead_scores + live_scores
        ranks = sample(scores, k)
        n = len(dead_scores)
        dead_scores = [dead_scores[r] for r in ranks if r < n]
        dead_samples = [dead_samples[r] for r in ranks if r < n]
        
        live_scores = [live_scores[r-n] for r in ranks if r >= n]
        live_samples = [live_samples[(r-n)//vocab_size]+[(r-n)%vocab_size] for r in ranks if r >= n]

        # live samples that should be dead are...
        # even if len(live_samples) == maxsample we dont want it dead because we want one
        # last prediction out of it to reach a headline of maxlenh
        def is_zombie(s):
            return s[-1] == eos or len(s) > maxsample
        
        # add zombies to the dead
        dead_scores += [c for s, c in zip(live_samples, live_scores) if is_zombie(s)]
        dead_samples += [s for s in live_samples if is_zombie(s)]
        
        # remove zombies from the living 
        live_scores = [c for s, c in zip(live_samples, live_scores) if not is_zombie(s)]
        live_samples = [s for s in live_samples if not is_zombie(s)]

    return dead_samples, dead_scores

In [ ]:
def keras_rnn_predict(samples, empty=empty, model=model, maxlen=maxlen):
    """for every sample, calculate probability for every possible label
    you need to supply your RNN model and maxlen - the length of sequences it can handle
    """
    sample_lengths = map(len, samples)
    assert all(l > maxlend for l in sample_lengths)
    assert all(l[maxlend] == eos for l in samples)
    # pad from right (post) so the first maxlend will be description followed by headline
    data = sequence.pad_sequences(samples, maxlen=maxlen, value=empty, padding='post', truncating='post')
    probs = model.predict(data, verbose=0, batch_size=batch_size)
    return np.array([output2probs(prob[sample_length - maxlend - 1]) for prob, sample_length in zip(probs, sample_lengths)])

In [ ]:
def vocab_fold(xs):
    """convert list of word indexes that may contain words outside vocab_size to words inside.
    If a word is outside, try first to use glove_idx2idx to find a similar word inside.
    If none exist then replace all accurancies of the same unknown word with <0>, <1>, ...
    """
    xs = [x if x < vocab_size - nb_unknown_words else glove_idx2idx.get(x, x) for x in xs]
    # the more popular word is <0> and so on
    outside = sorted([x for x in xs if x >= vocab_size - nb_unknown_words])
    # if there are more than nb_unknown_words oov words then put them all in nb_unknown_words-1
    outside = dict((x, vocab_size - 1 - min(i, nb_unknown_words - 1)) for i, x in enumerate(outside))
    xs = [outside.get(x, x) for x in xs]
    return xs

In [ ]:
def vocab_unfold(desc, xs):
    # assume desc is the unfolded version of the start of xs
    unfold = {}
    for i, unfold_idx in enumerate(desc):
        fold_idx = xs[i]
        if fold_idx >= vocab_size - nb_unknown_words:
            unfold[fold_idx] = unfold_idx
    return [unfold.get(x, x) for x in xs]

In [ ]:
import sys
import Levenshtein

def gensamples(X=None, X_test=None, Y_test=None, avoid=None, avoid_score=1, skips=2, k=10, batch_size=batch_size, short=True, temperature=1., use_unk=True):
    if X is None or isinstance(X, int):
        if X is None:
            i = random.randint(0, len(X_test) - 1)
        else:
            i = X
        print 'HEAD %d:'%i,' '.join(idx2word[w] for w in Y_test[i])
        print 'DESC:',' '.join(idx2word[w] for w in X_test[i])
        sys.stdout.flush()
        x = X_test[i]
    else:
        x = [word2idx[w.rstrip('^')] for w in X.split()]
        
    if avoid:
        # avoid is a list of avoids. Each avoid is a string or list of word indeicies
        if isinstance(avoid, str) or isinstance(avoid[0], int):
            avoid = [avoid]
        avoid = [a.split() if isinstance(a, str) else a for a in avoid]
        avoid = [vocab_fold([w if isinstance(w, int) else word2idx[w] for w in a])
                 for a in avoid]

    print 'HEADS:'
    samples = []
    if maxlend == 0:
        skips = [0]
    else:
        skips = range(min(maxlend, len(x)), max(maxlend, len(x)), abs(maxlend - len(x)) // skips + 1)
    for s in skips:
        start = lpadd(x[:s])
        fold_start = vocab_fold(start)
        sample, score = beamsearch(predict=keras_rnn_predict, start=fold_start, avoid=avoid, avoid_score=avoid_score,
                                   k=k, temperature=temperature, use_unk=use_unk)
        assert all(s[maxlend] == eos for s in sample)
        samples += [(s, start, scr) for s, scr in zip(sample, score)]

    samples.sort(key=lambda x: x[-1])
    codes = []
    for sample, start, score in samples:
        code = ''
        words = []
        sample = vocab_unfold(start, sample)[len(start):]
        for w in sample:
            if w == eos:
                break
            words.append(idx2word[w])
            code += chr(w//(256*256)) + chr((w//256)%256) + chr(w%256)
        if short:
            distance = min([100] + [-Levenshtein.jaro(code, c) for c in codes])
            if distance > -0.6:
                print score, ' '.join(words)
        #         print '%s (%.2f) %f'%(' '.join(words), score, distance)
        else:
                print score, ' '.join(words)
        codes.append(code)
    return samples

### Example Testing

In [ ]:
seed = 8
random.seed(seed)
np.random.seed(seed)

In [ ]:
samples = gensamples(X=X, skips=2, batch_size=batch_size, k=10, temperature=1.)

In [ ]:
X = "18 Cake GIFs That 'll Make You Moist"
Y = "Is it 350degF^ in here or is it just me ?"

In [ ]:
samples = gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1.)

In [ ]:
X = "President Barack Obama 's re-election campaign is fundraising off of comments on Obama 's birth certificate by Mitt Romney 's son Matt ."

In [ ]:
gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1, use_unk=True, short=False);

# Weights

In [ ]:
def wsimple_context(X, mask, n=activation_rnn_size, maxlend=maxlend, maxlenh=maxlenh):
    desc, head = X[:, :maxlend], X[:, maxlend:]
    head_activations, head_words = head[:, :, :n], head[:, :, n:]
    desc_activations, desc_words = desc[:, :, :n], desc[:, :, n:]
    
    # RTFM http://deeplearning.net/software/theano/library/tensor/basic.html#theano.tensor.batched_tensordot
    # activation for every head word and every desc word
    activation_energies = K.batch_dot(head_activations, desc_activations, axes=([2],[2]))
    # make sure we dont use description words that are masked out
    assert mask.ndim == 2
    activation_energies = K.switch(mask[:, None, :maxlend], activation_energies, -1e20)
    
    # for every head word compute weights for every desc word
    activation_energies = K.reshape(activation_energies,(-1,maxlend))
    activation_weights = K.softmax(activation_energies)
    activation_weights = K.reshape(activation_weights,(-1,maxlenh,maxlend))

    return activation_weights


class WSimpleContext(Lambda):
    def __init__(self):
        super(WSimpleContext, self).__init__(wsimple_context)
        self.supports_masking = True

    def compute_mask(self, input, input_mask=None):
        return input_mask[:, maxlend:]
    
    def get_output_shape_for(self, input_shape):
        nb_samples = input_shape[0]
        n = 2*(rnn_size - activation_rnn_size)
        return (nb_samples, maxlenh, n)

In [ ]:
wmodel = Sequential()
wmodel.add(rnn_model)

In [ ]:
wmodel.add(Lambda(wsimple_context,
                 mask = lambda inputs, mask: mask[:,maxlend:],
                 output_shape = lambda input_shape: (input_shape[0], maxlenh, maxlend)))

In [ ]:
wmodel.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Test with Weight Update Model

In [ ]:
seed = 8
random.seed(seed)
np.random.seed(seed)

In [ ]:
X = "Representatives of the groups depicted in The Revenant^ spoke with BuzzFeed News about the actor 's Golden Globes speech calling on listeners to `` protect ... indigenous lands . ''"
Y = "Native American Groups Officially Respond To Leonardo DiCaprio 's Call To Action"

In [ ]:
samples = gensamples(X, skips=2, batch_size=batch_size, k=10, temperature=1.)

In [ ]:
sample = samples[0][0]

In [ ]:
' '.join([idx2word[w] for w in sample])

In [ ]:
data = sequence.pad_sequences([sample], maxlen=maxlen, value=empty, padding='post', truncating='post')
data.shape

In [ ]:
weights = wmodel.predict(data, verbose=0, batch_size=1)
weights.shape

In [ ]:
startd = np.where(data[0,:] != empty)[0][0]
lenh = np.where(data[0,maxlend+1:] == eos)[0][0]
startd, lenh

### Plot weights and values

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(np.array(weights[0,:lenh,startd:].flatten()+1), bins=100);

### Create heat map for headline words versus description words

In [ ]:
import numpy as np
from IPython.core.display import display, HTML

def heat(sample,weights,dark=0.3):
    weights = (weights - weights.min())/(weights.max() - weights.min() + 1e-4)
    html = ''
    fmt = ' <span style="background-color: #{0:x}{0:x}ff">{1}</span>'
    for t,w in zip(sample,weights):
        c = int(256*((1.-dark)*(1.-w)+dark))
        html += fmt.format(c,idx2word[t])
    display(HTML(html))

In [ ]:
heat(sample, weights[0,-1])

In [ ]:
columns = [idx2word[data[0,i]] for i in range(startd,maxlend)]
rows = [idx2word[data[0,i]] for i in range(maxlend+1,maxlend+lenh+1)]

In [ ]:
df = pd.DataFrame(weights[0,:lenh,startd:],columns=columns,index=rows)

In [ ]:
sns.heatmap(df);